In [1]:
from google.colab import drive 
drive.mount('/content/drive')
%cd drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [2]:
!pip install uvicorn
!pip install fastapi
!pip install nest_asyncio
!pip install pystan
!pip install prophet
!pip install pyngrok 
!pip install pmdarima
!pip install -v scikit-learn==0.23.2

Created temporary directory: /tmp/pip-ephem-wheel-cache-qlfm22tq
Created temporary directory: /tmp/pip-req-tracker-p72c1qxz
Created requirements tracker '/tmp/pip-req-tracker-p72c1qxz'
Created temporary directory: /tmp/pip-install-r1jwi6ue
Cleaning up...
Removed build tracker '/tmp/pip-req-tracker-p72c1qxz'


In [2]:
# for FastAPI
from fastapi import FastAPI
import uvicorn
import pydantic
from pyngrok import ngrok
import nest_asyncio
# for FBprophet
from datetime import *
import pandas_datareader as pdr
import numpy as np
import pandas as pd
import holidays
from prophet import Prophet
# for arima
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
# for LSTM
 
from tensorflow.keras.models import load_model
import pickle
# for transformer
from utils.Time2Vector import Time2Vector
from utils.Attention import MultiAttention, SingleAttention
from utils.Encoder import TransformerEncoder
from tensorflow import keras
import warnings
nest_asyncio.apply()

In [3]:
def indicator(reg_pred,arima_pred,lstm_pred,prophet_pred,trans_pred,real):
  '''
    Take Decision based on specific formula 
    Args:
        (float) Regression prediciton - the predcition of Ensemble Regression Models
        (float) Arima prediciton - the predcition of Arima Model
        (float) LSTM prediciton - the predcition of LSTM Model
        (float) Prophet prediciton - the predcition of Prophet Model
        (float) Transformer prediciton - the predcition of Tranformer Model
        (float) Real - the value of Close in the last day
  Returns:
      (Array) Decision -array of values of [-1,0,1] 
  '''
  # Fromula
  percent=real*0.5/100

  predictions=[reg_pred,arima_pred,lstm_pred,prophet_pred,trans_pred]
  
  decisions=[]
  for i in predictions:
 
    if i>real+percent:
      action=1

    elif i<real-percent:
      action=-1

    else:
      action=0
    
    decisions.append(action)
  
  return decisions

In [4]:
import numpy as np

def d3_scale(data, out_range=(0, 1)):
  '''
    Scale values from 0 to 1  
    Args:
        (float) data - The unscaled data 
        (tuple) data - The range of the new scaled data 
        
  Returns:
      (Array) array that has values from 0 into 1  '''
  domain = [np.min(data, axis=0), np.max(data, axis=0)]

  def interp(x):
      return out_range[0] * (1.0 - x) + out_range[1] * x

  def uninterp(x):
      b = 0
      if (domain[1] - domain[0]) != 0:
          b = domain[1] - domain[0]
      else:
          b =  1.0 / domain[1]
      return (x - domain[0]) / b

  return interp(uninterp(data))



In [5]:

def prophet (ticker):
  """
  Forcasting using prophet ! by Getting the desired data from yahoo, then doing some data manipulation, then the comes the prophet's turn
  Args:
      (str) ticket - the ticker of desired dataset (company)
  Returns:
      (float) prophet_output - the model out-put (the prediction of the next day)
  """

  # data_gathering
  df = pdr.DataReader(ticker, data_source='yahoo', start='2015-01-01')

  # data manipulation
  holiday = pd.DataFrame([])
  for date, name in sorted(holidays.UnitedStates(years=[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]).items()):
      holiday = holiday.append(pd.DataFrame({'ds': date, 'holiday': "US-Holidays"}, index=[0]), ignore_index=True)
  holiday['ds'] = pd.to_datetime(holiday['ds'], format='%Y-%m-%d', errors='ignore')

  # data frame modification to be accepted by prophet
  data = df['Close'].reset_index()
  data.columns = ['ds', 'y']

  # model building
  m = Prophet(holidays=holiday,seasonality_mode='additive', changepoint_prior_scale = 0.1, seasonality_prior_scale=0.01)
  m.fit(data)

  # model predictions
  future = m.make_future_dataframe(periods=1)
  model_prediction = m.predict(future) 
  prophet_prediction = float(model_prediction[ 'yhat'][-1:])
  return prophet_prediction


def arima(ticker):
  """
  Forcasting using ARIMA ! by Getting the desired data from yahoo, 
  then finding the best order of arima params then the comes the ARIMA's turn
  Args:
      (str) ticket - the ticker of desired dataset (company)
  Returns:
      (float) arima_output - the model out-put (the prediction of the next day)
      (float) diff - the model output - today's price (the diff between tomorrow's prediction and today's real value)
  """
    
  # data gathering
  df = pdr.DataReader(ticker, data_source='yahoo', start='2016-01-01')
  df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
  df = pd.Series(df['Close'])
  last_day=df[-1]

  # finding the best order
  auto_order = pm.auto_arima(df, start_p=0, start_q=0, test='adf', max_p=3, max_q=3, m=1,d=None,seasonal=False   
                    ,start_P=0,D=0, trace=True,error_action='ignore',suppress_warnings=True,stepwise=True)
  best_order = auto_order.order

  # model fitting
  model = ARIMA(df, order=best_order)
  model_fit = model.fit(disp=0)
  arima_prediction ,se, conf = model_fit.forecast(1)
  
  diff = arima_prediction - last_day
  
  return arima_prediction , diff


def lstm(data_set):
  """
  Getting the desired data from yahoo, then doing some data manipulation such as data
  reshaping
  Args:
      (str) data_set - the ticker of desired dataset (company)
  Returns:
      (float) diff_prediction - the model out-put (the prediction of the next day)
      (float) real_prediction - the model output + today's price (real price of tomorrow)
  """

  # data gathering
  df = pdr.DataReader(data_set, data_source='yahoo', start=date.today() - timedelta(100))

  # data manipulation

  # creating a new df with Xt - Xt-1 values of the close prices (most recent 60 days)
  close_df = df['2012-01-01':].reset_index()['Close'][-61:]
  close_diff = close_df.diff().dropna()
  data = np.array(close_diff).reshape(-1, 1)

  # reshaping the data to 3D to be accepted by our LSTM model
  model_input = np.reshape(data, (1, 60, 1))

  # loading the model and predicting
  loaded_model = load_model("lstm_f_60.hdf5")
  model_prediction = float(loaded_model.predict(model_input))
  real_prediction = model_prediction + df['Close'][-1]
  

  return model_prediction, real_prediction


def Regression(ticker):
  """
  Forcasting using an ensambled model between SVR, Ridge and Linear regression! by Getting the desired data from yahoo, 
  then doing some data manipulation
  Args:
      (str) ticket - the ticker of desired dataset (company)
  Returns:
      (float) arima_output - the model out-put (the prediction of the next day)
      (float) diff - the model output - today's price (the diff between tomorrow's prediction and today's real value)
  """
  start_date = datetime.now() - timedelta(7)
  start_date = datetime.strftime(start_date, '%Y-%m-%d')

  df = pdr.DataReader(ticker, data_source='yahoo', start=start_date)  # read data
  df.drop('Volume', axis='columns', inplace=True)

  if df.index[-1]==datetime.now().strftime('%Y-%m-%d') :
    df=df.iloc[-2:]
  else :
    df =df.iloc[-1:]
  X = df[['High', 'Low', 'Open', 'Adj Close']]  # input columns
  y = df[['Close']]  # output column
  input = X
  loaded_model = pickle.load(open('regression_model.pkl', 'rb'))
  reg_prediction = loaded_model.predict(input)
  reg_diff=reg_prediction-df.Close[-1]

  return  reg_prediction,reg_diff

def Transformer(ticker):
  seq_len = 32

  start_date = datetime.now() - timedelta(48)
  start_date = datetime.strftime(start_date, '%Y-%m-%d')

  df = pdr.DataReader(ticker, data_source='yahoo', start=start_date)

  df.drop('Volume', axis=1, inplace=True)

  # df[df.columns] = scaler.fit_transform(df)
  df = df[['High', 'Low', 'Open', 'Adj Close', 'Close']]

  '''Create training, validation and test split'''

  test_data = df.values

  # Test data
  X_test, y_test = [], []
  for i in range(seq_len, len(test_data)):
      X_test.append(test_data[i - seq_len:i])
      y_test.append(test_data[:, 4][i])
  X_test, y_test = np.array(X_test), np.array(y_test)

  custom_objects = {"Time2Vector": Time2Vector,
                    "MultiAttention": MultiAttention,
                    'TransformerEncoder': TransformerEncoder}
  with keras.utils.custom_object_scope(custom_objects):
      final_model = load_model('Transformer+TimeEmbedding.hdf5')

  trans_prediction = float(final_model.predict(X_test)[-1])
  trans_difference = trans_prediction - df.Close[-1]

  return trans_prediction, trans_difference

def real (ticker):
  
  start_date = datetime.now() - timedelta(10)
  start_date = datetime.strftime(start_date, '%Y-%m-%d')

  df = pdr.DataReader(ticker, data_source='yahoo', start=start_date)  # read data
  df.drop('Volume', axis='columns', inplace=True)

  if df.index[-1]==datetime.now().strftime('%Y-%m-%d') :
    close=df.Close[-2]
  
  else :
    close =df.Close[-1]
  
  return close
 
 

In [6]:
app = FastAPI()


@app.get('/')
def index():
    return {'message': 'This is your fav stock predictor!'}


@app.post('/predict')
async def predict_price(data: str):
    if data == 'F':
      prophet_prediction = float(prophet(data))
      arima_prediction, diff = arima(data)
      model_prediction, lstm_prediction = lstm(data)
      reg_prediction,reg_diff = Regression(data)
      trans_prediction, trans_difference = Transformer(data)
      close=real(data)
      
      indicators=np.array(indicator(reg_prediction[0],arima_prediction[0],lstm_prediction,prophet_prediction,trans_prediction,close))
      weights=np.array([0.3,0.3,0.3,0.05,0.05])
      
      
      final_decision=indicators*weights
      final_decision=final_decision.sum()
      final_decision=final_decision.item(0)
      decision_array=np.array([-1,final_decision,1],dtype=np.float)
      decision_array=d3_scale(decision_array)
      final_decision=round(decision_array[1],1)*10


      

      return {
        'Decision': final_decision
        
            }

    else:
      return {"the ticker not supported yet"}

    

# App startup
ngrok_tunnel = ngrok.connect(8090)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8090)

Public URL: http://3839cc4e16d2.ngrok.io


INFO:     Started server process [939]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8090 (Press CTRL+C to quit)


INFO:     154.181.127.172:0 - "GET / HTTP/1.1" 200 OK
INFO:     154.181.127.172:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     154.181.127.172:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.181.127.172:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-490.809, Time=0.26 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-488.811, Time=0.09 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-488.811, Time=0.33 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-492.791, Time=0.09 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-486.809, Time=0.35 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.140 seconds


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/arima_model.py:472: FutureWarning:


statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



INFO:     154.181.127.172:0 - "POST /predict?data=F HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2021-06-28T21:55:45+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [939]
INFO:uvicorn.error:Finished server process [939]
